<a href="https://colab.research.google.com/github/RodolfoFerro/deep-solar/blob/main/notebooks/Solution_(our_approach).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solution (our approach)

## Current pipeline method

At present, the raw data are converted to physics units using a calibration table, summed over the sensor segments, and recast as phase space distribution functions. A variety of ad hoc operations are invoked in order to tweak the background subtraction, isolate the physically relevant sub-range in voltage, and remove transients. Then a Gaussian peak fitting is performed. The parameters of the Gaussian map directly to _(n, w, |v|)_. Finally, the ratios between the signal peak values are fed into a table lookup to estimate the flow angle and complete the velocity vector.

## Our approach

This second model is trained on a compressed representation of data. The raw data corresponding to coordinates in `BGS` format was converted into a single Real Number value using the norm of the vector.

> **Notes:**
> - **BGSE:** Geocentric Solar Ecliptic system. This has its X-axis pointing from the Earth toward the Sun and its Y-axis is chosen to be in the ecliptic plane pointing towards dusk (thus opposing planetary motion). Its Z-axis is parallel to the ecliptic pole. Relative to an inertial system this system has a yearly rotation.
> - **BGSM:** Geocentric Solar Magnetospheric system. This has its X-axis from the Earth to the Sun. The Y-axis is defined to be perpendicular to the Earth's magnetic dipole so that the X-Z plane contains the dipole axis. The positive Z-axis is chosen to be in the same sense as the northern magnetic pole. The difference between the GSM and GSE systems is simply a rotation about the X-axis.

This led us to have a 2-column dataset, with the first column corresponding to the mentioned norm of the coordinates, and with the second column corresponding to BF1 values from the Wind dataset.

In [1]:
!pip install wget cdflib dtw-python -q

     |████████████████████████████████| 72 kB 1.3 MB/s 
     |████████████████████████████████| 633 kB 28.5 MB/s 


> **Notes:** 
> - Data from [_Wind dataset_](https://cdaweb.gsfc.nasa.gov/pub/data/wind/mfi/mfi_h2/2022/) is available from 1994 (1994-11-13) to 2022 (2022-09-17).
> - Data form [_DSCOVR magnetic field dataset_](https://cdaweb.gsfc.nasa.gov/pub/data/dscovr/h0/mag/2022/) is available from 2015 (2015-06-08) to 2022 (2022-09-17).

We will start loading some libraries:

In [10]:
from datetime import datetime
from datetime import timedelta
import re

import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
import pandas as pd
import xarray as xr
from dtw import dtw
from dtw import rabinerJuangStepPattern
import cdflib
import wget

plt.style.use('seaborn')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import sys


We will no proceed to create a date range estimator, in order to dowload all files between that range for training.

In [4]:
def date_range(start, end):
    delta = end - start  # as timedelta
    days = [start + timedelta(days=i) for i in range(delta.days + 1)]
    return days

We download the files:

In [13]:
start_date = datetime(2022, 8, 17)
end_date = datetime(2022, 9, 17)

dates = date_range(start_date, end_date)
dates = [date.strftime('%Y%m%d') for date in dates]
files = []

for date in tqdm(dates):
    for v in ['04', '03', '02', '01']:
        try:
            wind_url = f'https://cdaweb.gsfc.nasa.gov/pub/data/wind/mfi/mfi_h2/2022/wi_h2_mfi_{date}_v{v}.cdf'
            wind_filename = wget.download(wind_url)
            print('[INFO] File found! ->', wind_filename)
            files.append(wind_filename)
            break
        except:
            continue

  0%|          | 0/32 [00:00<?, ?it/s]

[INFO] File found! -> wi_h2_mfi_20220817_v04.cdf
[INFO] File found! -> wi_h2_mfi_20220818_v04.cdf
[INFO] File found! -> wi_h2_mfi_20220819_v04.cdf
[INFO] File found! -> wi_h2_mfi_20220820_v04.cdf
[INFO] File found! -> wi_h2_mfi_20220821_v04.cdf
[INFO] File found! -> wi_h2_mfi_20220822_v04.cdf
[INFO] File found! -> wi_h2_mfi_20220823_v04.cdf
[INFO] File found! -> wi_h2_mfi_20220824_v04.cdf
[INFO] File found! -> wi_h2_mfi_20220825_v04.cdf
[INFO] File found! -> wi_h2_mfi_20220826_v04.cdf
[INFO] File found! -> wi_h2_mfi_20220827_v04.cdf
[INFO] File found! -> wi_h2_mfi_20220828_v04.cdf
[INFO] File found! -> wi_h2_mfi_20220829_v04.cdf
[INFO] File found! -> wi_h2_mfi_20220830_v04.cdf
[INFO] File found! -> wi_h2_mfi_20220831_v04.cdf
[INFO] File found! -> wi_h2_mfi_20220901_v04.cdf
[INFO] File found! -> wi_h2_mfi_20220902_v04.cdf
[INFO] File found! -> wi_h2_mfi_20220903_v04.cdf
[INFO] File found! -> wi_h2_mfi_20220904_v04.cdf
[INFO] File found! -> wi_h2_mfi_20220905_v04.cdf
[INFO] File found! -

### Data loading

To load data, we will use [`cfdlib`](https://cdflib.readthedocs.io/en/stable/), which will allow us to interact with data as XArray tables.


We will iteratively load data and append it to a big table.

> **CONSIDER:** If you are running this notebook, this process takes a while.

In [14]:
df = pd.DataFrame()

for wind_file in tqdm(files):
    print("[INFO] Processing file ->", wind_file)
    wind_cdf_data = cdflib.cdf_to_xarray(wind_file, to_datetime=True, fillval_to_nan=True)

    wind_data = wind_cdf_data['BGSE'].to_pandas()
    wind_data.columns = ['x', 'y', 'z']
    wind_data['BF1'] = wind_cdf_data['BF1'].to_pandas()
    wind_data['norm'] = np.linalg.norm(wind_data[['x', 'y', 'z']].values, axis=1)

    df = df.append(wind_data)

  0%|          | 0/32 [00:00<?, ?it/s]

[INFO] Processing file -> wi_h2_mfi_20220817_v04.cdf
[INFO] Processing file -> wi_h2_mfi_20220818_v04.cdf
[INFO] Processing file -> wi_h2_mfi_20220819_v04.cdf
[INFO] Processing file -> wi_h2_mfi_20220820_v04.cdf
[INFO] Processing file -> wi_h2_mfi_20220821_v04.cdf
[INFO] Processing file -> wi_h2_mfi_20220822_v04.cdf
[INFO] Processing file -> wi_h2_mfi_20220823_v04.cdf
[INFO] Processing file -> wi_h2_mfi_20220824_v04.cdf
[INFO] Processing file -> wi_h2_mfi_20220825_v04.cdf
[INFO] Processing file -> wi_h2_mfi_20220826_v04.cdf
[INFO] Processing file -> wi_h2_mfi_20220827_v04.cdf
[INFO] Processing file -> wi_h2_mfi_20220828_v04.cdf
[INFO] Processing file -> wi_h2_mfi_20220829_v04.cdf
[INFO] Processing file -> wi_h2_mfi_20220830_v04.cdf
[INFO] Processing file -> wi_h2_mfi_20220831_v04.cdf
[INFO] Processing file -> wi_h2_mfi_20220901_v04.cdf
[INFO] Processing file -> wi_h2_mfi_20220902_v04.cdf
[INFO] Processing file -> wi_h2_mfi_20220903_v04.cdf
[INFO] Processing file -> wi_h2_mfi_20220904_v

As we can see, for a month of information, we have ~29.7 million rows.

In [16]:
len(df)

29732420

### Data preparation

We will now proceed to take 2 of the features, a representation of `BGSE` and `BF1` and scale the data, prior to train a model.

In [17]:
from sklearn.preprocessing import MinMaxScaler


features = df[['norm', 'BF1']]
scaler = MinMaxScaler()
scaler.fit(features)
scaled_features = scaler.transform(features)

In [18]:
scaled_features

array([[0.3837791 , 0.3837791 ],
       [0.38262427, 0.38262424],
       [0.3814674 , 0.3814674 ],
       ...,
       [0.54184467, 0.54184467],
       [0.54116344, 0.54116344],
       [0.5404409 , 0.5404409 ]], dtype=float32)

### Model training

We proceed to create and train an autoencoder.

> **CONSIDER:** If you are running this notebook, this process takes a while.

In [19]:
# Model and performance
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


data_length = 2

w_train, w_test = train_test_split(scaled_features, test_size=0.2, random_state=42)

input = tf.keras.layers.Input(shape=(data_length,))

# Encoder layers
encoder = tf.keras.Sequential([
  tf.keras.layers.Dense(16, activation='relu'),
  tf.keras.layers.Dense(8, activation='relu'),
  tf.keras.layers.Dense(4, activation='relu')])(input)

# Decoder layers
decoder = tf.keras.Sequential([
      tf.keras.layers.Dense(8, activation='relu'),
      tf.keras.layers.Dense(16, activation='relu'),
      tf.keras.layers.Dense(data_length, activation='sigmoid')])(encoder)

# Create the autoencoder
autoencoder = tf.keras.Model(inputs=input, outputs=decoder)

Once the model is instantiated, we proceed to compile it and train it for 20 epochs (100 epochs take around 18 hours to train).

> **CONSIDER:** If you are running this notebook, this process takes a while.

In [ ]:
# Compile the autoencoder
autoencoder.compile(optimizer='adam', loss='msle',  metrics=['mse'])

# Fit the autoencoder
history = autoencoder.fit(
    w_train,
    w_train,
    epochs=20,
    batch_size=64,
    validation_data=(w_test, w_test))

Epoch 1/20
371656/371656 [==============================] - 708s 2ms/step - loss: 1.5795e-06 - mse: 2.7071e-06 - val_loss: 1.1704e-06 - val_mse: 1.9254e-06
Epoch 2/20
371656/371656 [==============================] - 604s 2ms/step - loss: 9.5320e-07 - mse: 1.6656e-06 - val_loss: 2.3232e-07 - val_mse: 4.3890e-07
Epoch 3/20
371656/371656 [==============================] - 564s 2ms/step - loss: 1.6904e-07 - mse: 3.5211e-07 - val_loss: 8.9965e-08 - val_mse: 1.9326e-07
Epoch 4/20
371656/371656 [==============================] - 565s 2ms/step - loss: 1.4246e-07 - mse: 2.9538e-07 - val_loss: 9.2353e-08 - val_mse: 1.9943e-07
Epoch 5/20
371656/371656 [==============================] - 566s 2ms/step - loss: 1.1309e-07 - mse: 2.2157e-07 - val_loss: 8.6929e-08 - val_mse: 1.5410e-07
Epoch 6/20
371656/371656 [==============================] - 559s 2ms/step - loss: 7.6772e-08 - mse: 1.5246e-07 - val_loss: 1.9313e-08 - val_mse: 4.7541e-08
Epoch 7/20
371656/371656 [==============================] - 590s

Let's see the training history:

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go


x = np.arange(len(history.history['loss']))

fig = make_subplots(rows=1, cols=1)
fig.append_trace(go.Scatter(x=x, y=history.history['loss'], name='loss'), row=1, col=1)
fig.append_trace(go.Scatter(x=x, y=history.history['val_loss'], name='val_loss'), row=1, col=1)
fig.update_layout(xaxis_title="Epoch", yaxis_title="Loss")
fig.show()

### Saving the model

Once the model is trained, we proceed to save it, so we can export it and use it in our application.

In [ ]:
# Serialize model tomodels JSON:
json_autoencoder = autoencoder.to_json()
with open('general_model.json', 'w') as json_file:
    json_file.write(json_autoencoder)

# Serialize weights to HDF5 (h5py needed):
autoencoder.save_weights('general_model.h5')
print('[INFO] Model saved to disk.')

## Model testing

In [ ]:
def find_threshold(model, x_train_scaled):
    reconstructions = model.predict(x_train_scaled)
    # provides losses of individual instances
    reconstruction_errors = tf.keras.losses.msle(reconstructions, x_train_scaled)
    # threshold for anomaly scores
    threshold = np.mean(reconstruction_errors.numpy()) \
        + 3 * np.std(reconstruction_errors.numpy())
    return threshold

def get_predictions(model, x_test_scaled, threshold):
    predictions = model.predict(x_test_scaled)
    # provides losses of individual instances
    errors = tf.keras.losses.msle(predictions, x_test_scaled)
    # 0 = anomaly, 1 = normal
    anomaly_mask = pd.Series(errors) > threshold
    preds = anomaly_mask.map(lambda x: 1 if x == True else 0)
    return errors, preds

In [ ]:
threshold = find_threshold(autoencoder, w_train)
print(f"Threshold: {threshold}")

In [ ]:
errors, predictions = get_predictions(autoencoder, w_test, threshold)

In [ ]:
reconstructions = autoencoder.predict(w_train)
reconstruction_errors = tf.keras.losses.msle(reconstructions, w_train)

In [ ]:
w_train[0], reconstructions[0], reconstruction_errors[0].numpy()

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

x = np.arange(50, 450)

fig = make_subplots(rows=2, cols=1)
fig.append_trace(go.Scatter(x=x, y=w_train[50:450, 1], line=dict(color='royalblue', width=4, dash='dot')), row=1, col=1)
fig.append_trace(go.Scatter(x=x, y=reconstructions[50:450, 1]), row=1, col=1)
fig.append_trace(go.Scatter(x=x, y=reconstruction_errors[50:450]), row=2, col=1)
fig.add_hline(y=threshold, row=2, col=1)
fig.show()